In [1]:
from __future__ import division
from __future__ import print_function

import argparse
import time

import numpy as np
import scipy.sparse as sp
import torch
import torchvision
from torch import optim
import pandas as pd

from gae.model import GCNModelVAE

from gae.optimizer import loss_function
from gae.utils import load_data, mask_test_edges, preprocess_graph, get_roc_score

import import_ipynb
from gaemain import gae_for

from scipy.stats.stats import pearsonr  
from scipy import spatial
import os
import glob
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

importing Jupyter notebook from gaemain.ipynb
importing Jupyter notebook from ram_mediapipe.ipynb


C:\Users\ramba\AppData\Local\Temp\ipykernel_11140\2195660888.py:22: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [2]:
def relu(x):
	return max(0.0, x)

In [3]:
def testing_latent_representation(dataset):
    testSets=[]
    personsPath="./datasets/"+dataset+"/"
    persons = os.listdir(personsPath)
    for person in persons:
        path="./datasets/"+dataset+"/"+person+"/test_image/"
        files=os.listdir(path)
        count=0

        cwd = os.getcwd()

        try:
            os.mkdir(cwd+"./datasets/"+dataset+"/"+person+"/"+person+"_testing_latent_representation")
        except OSError as error:
            # toDelete = cwd+"/orl_dataset/person"+str(person)+"/person"+str(person)+"_testing_latent_representation"
            filestoDelete = glob.glob(cwd+"./datasets/"+dataset+"/"+person+"/"+person+"_testing_latent_representation/*")
            for f in filestoDelete:
                os.remove(f)

        for image in files:
            image_path=path+image
            imageName=image.split(".")
            # print(image_path)
            count=count+1
            Z=gae_for(image_path)
            Z=Z.detach().numpy() #convert to Numpy array
            Z = pd.DataFrame(Z) #convert to a dataframe
            testSets.append(Z)
            csv_path="./datasets/"+dataset+"/"+person+"/"+person+"_testing_latent_representation/"+imageName[0]+"_signature.csv"
            Z.to_csv(csv_path,index=False) #save to file
        print("Latent representation of ",person," is completed")
    print("total ",len(testSets)," test cases")

In [4]:
def fetching_testing_latent(dataset, ind, testSets, testPersonName):
    personsPath="./datasets/"+dataset+"/"
    persons = os.listdir(personsPath)
    for person in persons:
        tests=glob.glob("./datasets/"+dataset+"/"+person+"/"+person+"_testing_latent_representation/*.csv")
        for test in tests: #for eacg file
            personS=test
            testSets.append(np.loadtxt(test,delimiter=','))  #sign[ind]= content of file, seperated by comma
            testSets[ind]=testSets[ind][1:]
            testPersonName.append(personS)
            ind+=1
    return testSets, testPersonName

In [5]:
def c_dist(dataset, testPersonName, testSets):  
  acc=[]
  avg_p=[]
  avg_recall=[]
  avg_fScore=[]
  betaSquare=0.5*0.5
  r_rate=0 #recognition rate
  pr_rate=0 #recognition rate
  acc=[]
  sign=[]  #15*null
  index=0
  count=1

  for testSet in testSets:
    data = []
    test=[]
    TP=0
    TN=0
    FP=0
    FN=0
    tdata=testSet
    dist=[]
    pearson_dist=[]
    cosine_dist=[]
    c_dist=[]
    orig_path="./datasets/"+dataset+"/"
    files=os.listdir(orig_path)
    tests=[]
    trainPersonName=[]
    numberOfTrainedRepresentations=[]
    for file in files: #for each person
      tests=glob.glob(orig_path + file + '/' + file +'_training_latent_representation/*.csv')
      numberOfTrainedRepresentations.append(len(tests))
      numberOfTrainedRepresentations2=len(tests)
      sign=[None]*15  #15*null
      ind=0
      for test in tests: #for eacg file
        personS2=test
        trainPersonName.append(personS2)
        sign[ind]=np.loadtxt(test,delimiter=',')  #sign[ind]= content of file, seperated by comma
        sign[ind]=sign[ind][1:]
        ind+=1
    # print(trainPersonName)
      for ele in sign:
        if ele is None:
          break
        c_dist.append(1/(1 + np.exp(-pearsonr(np.array(tdata).flatten() ,np.array(ele).flatten())[0])))

    max_c_dist=c_dist.index(max(c_dist))
    print("Test case: ",count)
    count=count+1
    # print("maximum c_dist=",max(c_dist))
    if(testPersonName[index].split("/")[3] != trainPersonName[max_c_dist//numberOfTrainedRepresentations2].split("/")[3]):
      print("\t\t\t\tthis is wrong")
      TP=0
      TN=13
      FP=1
      FP=1
    elif(testPersonName[index].split("/")[3] == trainPersonName[max_c_dist//numberOfTrainedRepresentations2].split("/")[3]):
      r_rate+=1
      TP=1
      TN=14
      FN=0
      FP=0

    # print("Printing test images")
    print("test subject name: ", testPersonName[index].split("/")[3])
    path1=testPersonName[index].split("/")[0]+"/"+testPersonName[index].split("/")[1]+"/"+testPersonName[index].split("/")[2]+"/"+testPersonName[index].split("/")[3]+"/test_image/"
    path2=testPersonName[index].split("\\")[-1].split("_signature.csv")[0]+".jpg"
    path=path1+path2
    # print(path1)
    # print(path2)
    image1 = mpimg.imread(path)

    # print("printing matched image")
    print("Matched subject name: ", trainPersonName[max_c_dist//numberOfTrainedRepresentations2].split("/")[3])
    path3=trainPersonName[max_c_dist//numberOfTrainedRepresentations2 ].split("/")[0]+"/"+trainPersonName[max_c_dist//numberOfTrainedRepresentations2 ].split("/")[1]+"/"+trainPersonName[max_c_dist//numberOfTrainedRepresentations2 ].split("/")[2]+"/"+trainPersonName[max_c_dist//numberOfTrainedRepresentations2 ].split("/")[3]+"/train_images/"
    path4=trainPersonName[max_c_dist//numberOfTrainedRepresentations2 ].split("\\")[-1].split("_signature.csv")[0]+".jpg"
    path5=path3+path4
    image2 = mpimg.imread(path5)

    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(image1)
    axarr[1].imshow(image2)
    plt.show()
    plt.close()

    Accuracy = (TP + TN )/ (TP + TN + FP + FN) #(all correct / all)
    Misclassification = (FP + FN )/ (TP + TN + FP + FN) #(all incorrect / all)
    Precision  = TP / (TP + FP) #(true positives / predicted positives)
    deno=TP+FP
    Sensitivity   = TP / (deno) #aka Recall (true positives / all actual positives)
    Specificity  =TN / (TN + FP) #(true negatives / all actual negatives)
    F1_deno=(Precision+Sensitivity)
    if(F1_deno==0):
      F1_Score=0
    else:
      F1_Score=(1+betaSquare)*((Precision*Sensitivity)/(betaSquare*F1_deno))
    acc.append(Accuracy)
    avg_p.append(Precision)
    avg_recall.append(Sensitivity)
    avg_fScore.append(F1_Score)
    index=index+1
  # print(r_rate)
  # print("Total Test Case: {}\nAverage Accuracy: {}\nRecognition Rate: {}\nAverage Precision: {}\nAverage Recall: {}\nAverage F0.5 Score: {}".format(len(acc),(sum(acc)/len(acc)),r_rate/len(acc),(sum(avg_p)/len(avg_p)),(sum(avg_recall)/len(avg_recall)),(sum(avg_fScore)/len(avg_fScore))))  
  return(len(acc),(sum(acc)/len(acc)),r_rate/len(acc),(sum(avg_p)/len(avg_p)),(sum(avg_recall)/len(avg_recall)),(sum(avg_fScore)/len(avg_fScore)))


In [6]:
def cosine_dist(dataset, testPersonName, testSets):
  acc=[]
  avg_p=[]
  avg_recall=[]
  avg_fScore=[]
  betaSquare=0.5*0.5
  r_rate=0 #recognition rate
  pr_rate=0 #recognition rate
  acc=[]
  sign=[]  #15*null
  index=0
  count=1

  for testSet in testSets:
    data = []
    test=[]
    TP=0
    TN=0
    FP=0
    FN=0
    tdata=testSet
    dist=[]
    pearson_dist=[]
    cosine_dist=[]
    c_dist=[]
    orig_path="./datasets/"+dataset+"/"
    files=os.listdir(orig_path)
    tests=[]
    trainPersonName=[]
    numberOfTrainedRepresentations=[]
    for file in files: #for each person
      tests=glob.glob(orig_path + file + '/' + file +'_training_latent_representation/*.csv')
      numberOfTrainedRepresentations.append(len(tests))
      numberOfTrainedRepresentations2=len(tests)
      sign=[None]*15  #15*null
      ind=0
      for test in tests: #for eacg file
        personS2=test
        trainPersonName.append(personS2)
        sign[ind]=np.loadtxt(test,delimiter=',')  #sign[ind]= content of file, seperated by comma
        sign[ind]=sign[ind][1:]
        ind+=1

      for ele in sign:
        if ele is None:
          break
        cosine_dist.append(relu((1 - spatial.distance.cosine(np.array(tdata).flatten() ,np.array(ele).flatten()))))

    max_cosine=cosine_dist.index(max(cosine_dist))
    print("Cosine distance of all comparisons: ",cosine_dist)
    count=count+1
    print("maximum max_cosine=",max(cosine_dist))
    
    if(testPersonName[index].split("/")[3] != trainPersonName[max_cosine//numberOfTrainedRepresentations2].split("/")[3]):
      # print("\t\t\t\tthis is wrong")
      TP=0
      TN=13
      FP=1
      FP=1
    elif(testPersonName[index].split("/")[3] == trainPersonName[max_cosine//numberOfTrainedRepresentations2].split("/")[3]):
      r_rate+=1
      TP=1
      TN=14
      FN=0
      FP=0

    # print("Printing test images")
    print("test subject name: ", testPersonName[index].split("/")[3])
    path1=testPersonName[index].split("/")[0]+"/"+testPersonName[index].split("/")[1]+"/"+testPersonName[index].split("/")[2]+"/"+testPersonName[index].split("/")[3]+"/test_image/"
    path2=testPersonName[index].split("\\")[-1].split("_signature.csv")[0]+".jpg"
    path=path1+path2
    # print(path1)
    # print(path2)
    image1 = mpimg.imread(path)

    # print("printing matched image")
    print("Matched subject name: ", trainPersonName[max_cosine//numberOfTrainedRepresentations2].split("/")[3])
    path3=trainPersonName[max_cosine//numberOfTrainedRepresentations2 ].split("/")[0]+"/"+trainPersonName[max_cosine//numberOfTrainedRepresentations2 ].split("/")[1]+"/"+trainPersonName[max_cosine//numberOfTrainedRepresentations2 ].split("/")[2]+"/"+trainPersonName[max_cosine//numberOfTrainedRepresentations2 ].split("/")[3]+"/train_images/"
    path4=trainPersonName[max_cosine//numberOfTrainedRepresentations2 ].split("\\")[-1].split("_signature.csv")[0]+".jpg"
    path5=path3+path4
    image2 = mpimg.imread(path5)

    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(image1)
    axarr[1].imshow(image2)
    plt.show()
    plt.close()
    #Measures

    Accuracy = (TP + TN )/ (TP + TN + FP + FN) #(all correct / all)
    Misclassification = (FP + FN )/ (TP + TN + FP + FN) #(all incorrect / all)
    Precision  = TP / (TP + FP) #(true positives / predicted positives)
    deno=TP+FP
    Sensitivity   = TP / (deno) #aka Recall (true positives / all actual positives)
    Specificity  =TN / (TN + FP) #(true negatives / all actual negatives)
    F1_deno=(Precision+Sensitivity)
    if(F1_deno==0):
      F1_Score=0
    else:
      F1_Score=(1+betaSquare)*((Precision*Sensitivity)/(betaSquare*F1_deno))
    acc.append(Accuracy)
    avg_p.append(Precision)
    avg_recall.append(Sensitivity)
    avg_fScore.append(F1_Score)
    index=index+1
  # print(r_rate)
  # print("Total Test Case: {}\nAverage Accuracy: {}\nRecognition Rate: {}\nAverage Precision: {}\nAverage Recall: {}\nAverage F0.5 Score: {}".format(len(acc),(sum(acc)/len(acc)),r_rate/len(acc),(sum(avg_p)/len(avg_p)),(sum(avg_recall)/len(avg_recall)),(sum(avg_fScore)/len(avg_fScore))))
  return(len(acc),(sum(acc)/len(acc)),r_rate/len(acc),(sum(avg_p)/len(avg_p)),(sum(avg_recall)/len(avg_recall)),(sum(avg_fScore)/len(avg_fScore)))


In [7]:
def pearson_dist(dataset, testPersonName, testSets):
  # Pearson
  acc=[]
  avg_p=[]
  avg_recall=[]
  avg_fScore=[]
  betaSquare=0.5*0.5
  r_rate=0 #recognition rate
  pr_rate=0 #recognition rate
  acc=[]
  sign=[]  #15*null
  index=0
  count=1

  for testSet in testSets:
    data = []
    test=[]
    TP=0
    TN=0
    FP=0
    FN=0
    tdata=testSet
    dist=[]
    pearson_dist=[]
    cosine_dist=[]
    c_dist=[]
    orig_path="./datasets/"+dataset+"/"
    files=os.listdir(orig_path)
    tests=[]
    trainPersonName=[]
    numberOfTrainedRepresentations=[]
    for file in files: #for each person
      tests=glob.glob(orig_path + file + '/' + file +'_training_latent_representation/*.csv')
      numberOfTrainedRepresentations.append(len(tests))
      numberOfTrainedRepresentations2=len(tests)
      sign=[None]*15  #15*null
      ind=0
      for test in tests: #for eacg file
        personS2=test
        trainPersonName.append(personS2)
        sign[ind]=np.loadtxt(test,delimiter=',')  #sign[ind]= content of file, seperated by comma
        sign[ind]=sign[ind][1:]
        ind+=1

      for ele in sign:
        if ele is None:
          break
        pearson_dist.append(1/(1 + np.exp(-(pearsonr(np.array(tdata).flatten() ,np.array(ele).flatten())[0]))))

    max_ind=pearson_dist.index(max(pearson_dist))
    # print("Test case: ",count)
    count=count+1
    # print("maximum max_ind=",max_ind)
    
    if(testPersonName[index].split("/")[3] != trainPersonName[max_ind//numberOfTrainedRepresentations2].split("/")[3]):
      # print("\t\t\t\tthis is wrong")
      TP=0
      TN=13
      FP=1
      FP=1
    elif(testPersonName[index].split("/")[3] == trainPersonName[max_ind//numberOfTrainedRepresentations2].split("/")[3]):
      r_rate+=1
      TP=1
      TN=14
      FN=0
      FP=0

    # print("Printing test images")
    # print("test subject name: ", testPersonName[index].split("\\")[-1:][0].split("_")[0].split(".")[0])
    # path1=testPersonName[index].split("/")[0]+"/"+testPersonName[index].split("/")[1]+"/"+testPersonName[index].split("/")[2]+"/"+testPersonName[index].split("/")[3]+"/test_image/"
    # path2=testPersonName[index].split("\\")[-1].split("_signature.csv")[0]+".jpg"
    # path=path1+path2
    # image = mpimg.imread(path)
    # plt.imshow(image)
    # plt.show()

    # print("printing matched image")
    # print("Matched subject name: ", trainPersonName[max_ind//numberOfTrainedRepresentations2 ].split("\\")[-1:][0].split("_")[0].split(".")[0])
    # path3=trainPersonName[max_ind//numberOfTrainedRepresentations2 ].split("/")[0]+"/"+trainPersonName[max_ind//numberOfTrainedRepresentations2 ].split("/")[1]+"/"+trainPersonName[max_ind//numberOfTrainedRepresentations2 ].split("/")[2]+"/"+trainPersonName[max_ind//numberOfTrainedRepresentations2 ].split("/")[3]+"/train_images/"
    # path4=trainPersonName[max_ind//numberOfTrainedRepresentations2 ].split("\\")[-1].split("_signature.csv")[0]+".jpg"
    # path5=path3+path4
    # image = mpimg.imread(path5)
    # plt.imshow(image)
    # plt.show()
    #Measures

    Accuracy = (TP + TN )/ (TP + TN + FP + FN) #(all correct / all)
    Misclassification = (FP + FN )/ (TP + TN + FP + FN) #(all incorrect / all)
    Precision  = TP / (TP + FP) #(true positives / predicted positives)
    deno=TP+FP
    Sensitivity   = TP / (deno) #aka Recall (true positives / all actual positives)
    Specificity  =TN / (TN + FP) #(true negatives / all actual negatives)
    F1_deno=(Precision+Sensitivity)
    if(F1_deno==0):
      F1_Score=0
    else:
      F1_Score=(1+betaSquare)*((Precision*Sensitivity)/(betaSquare*F1_deno))
    acc.append(Accuracy)
    avg_p.append(Precision)
    avg_recall.append(Sensitivity)
    avg_fScore.append(F1_Score)
    index=index+1
  # print(r_rate)
  # print("Total Test Case: {}\nAverage Accuracy: {}\nRecognition Rate: {}\nAverage Precision: {}\nAverage Recall: {}\nAverage F0.5 Score: {}".format(len(acc),(sum(acc)/len(acc)),r_rate/len(acc),(sum(avg_p)/len(avg_p)),(sum(avg_recall)/len(avg_recall)),(sum(avg_fScore)/len(avg_fScore))))
  return(len(acc),(sum(acc)/len(acc)),r_rate/len(acc),(sum(avg_p)/len(avg_p)),(sum(avg_recall)/len(avg_recall)),(sum(avg_fScore)/len(avg_fScore)))


In [8]:
def test_controller(dataset):
    global testSets
    testSets=[]
    ind=0
    global testPersonName
    testPersonName=[]
    testing_latent_representation(dataset)
    testSets, testPersonName=fetching_testing_latent(dataset, ind, testSets, testPersonName)
    # print(testPersonName)
    # print(testPersonName)
    # c_dist_report = c_dist(dataset, testPersonName, testSets)
    cosine_report = cosine_dist(dataset, testPersonName, testSets)
    pearson_report = pearson_dist(dataset, testPersonName, testSets)
    return cosine_report, pearson_report

In [9]:
# print(test_controller("orl_dataset"))